In [1]:
import pandas as pnd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

Cargamos el csv para empezar el trabajo

In [2]:
mensajesTwitter = pnd.read_csv("datas/calentamientoClimatico_sin_preparar.csv", delimiter=";")
mensajesTwitter.head(10)

,Column1,Column2,Column3
0,tweet,existence,existence.confidence
1,Global warming report urges governments to act...,Yes,1
2,Fighting poverty and global warming in Africa ...,Yes,1
3,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786
4,Carbon offsets: How a Vatican forest failed to...,Yes,1
5,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes,0.8087
6,RT @sejorg: RT @JaymiHeimbuch: Ocean Saltiness...,Yes,1
7,Global warming evidence all around us|A messag...,Yes,1
8,Migratory Birds' New Climate Change Strategy: ...,Yes,1
9,Southern Africa: Competing for Limpopo Water: ...,Yes,1


Vemos la dimensión del dataset

In [3]:
mensajesTwitter.shape

(6091, 3)

Eliminamos la primera fila que no tiene ninguna utilidad

In [4]:
mensajesTwitter = mensajesTwitter.drop([0],axis=0)

Cambiamos el nombre de la columnas para reconocer cada una de ellas

In [5]:
column_names=['TWEET', 'CREENCIA', 'CONFIANZA']
mensajesTwitter.columns = column_names
mensajesTwitter

,TWEET,CREENCIA,CONFIANZA
1,Global warming report urges governments to act...,Yes,1
2,Fighting poverty and global warming in Africa ...,Yes,1
3,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786
4,Carbon offsets: How a Vatican forest failed to...,Yes,1
5,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes,0.8087
...,...,...,...
6086,"@bloodless_coup ""The phrase 'global warming' s...",Yes,1
6087,Virginia to Investigate Global Warming Scienti...,NaN,1
6088,Global warming you tube parody you will enjoy ...,No,0.6411
6089,One-Eyed Golfer: Don't dare tell me about glob...,No,1


Miramos a ver si tenemos algun valor nulo

In [6]:
mensajesTwitter.isna().sum()

TWEET           0
CREENCIA     1865
CONFIANZA       3
dtype: int64

Eliminamos las filas con valores nulos

In [7]:
mensajesTwitter = mensajesTwitter.dropna()
mensajesTwitter

,TWEET,CREENCIA,CONFIANZA
1,Global warming report urges governments to act...,Yes,1
2,Fighting poverty and global warming in Africa ...,Yes,1
3,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786
4,Carbon offsets: How a Vatican forest failed to...,Yes,1
5,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes,0.8087
...,...,...,...
6085,It's 83�_� and climbing in NYC. August weather...,Yes,1
6086,"@bloodless_coup ""The phrase 'global warming' s...",Yes,1
6088,Global warming you tube parody you will enjoy ...,No,0.6411
6089,One-Eyed Golfer: Don't dare tell me about glob...,No,1


Verificamos, una vez realizada la limpieza de nulos, que ya no queda ninguno

In [8]:
mensajesTwitter.isna().sum()

TWEET        0
CREENCIA     0
CONFIANZA    0
dtype: int64

Vemos los valores únicos que tienen las columnas

In [9]:
mensajesTwitter['CREENCIA'].unique()

array(['Yes', 'No'], dtype=object)

Empezamos con el proceso de lectura, para ello descargamos las librerías

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/alex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Categorizamos la columna CREENCIA 

In [11]:
mensajesTwitter['CREENCIA'] = (mensajesTwitter['CREENCIA']=='Yes').astype(int)
print(mensajesTwitter.head(100))

                                                 TWEET  CREENCIA CONFIANZA
1    Global warming report urges governments to act...         1         1
2    Fighting poverty and global warming in Africa ...         1         1
3    Carbon offsets: How a Vatican forest failed to...         1    0.8786
4    Carbon offsets: How a Vatican forest failed to...         1         1
5    URUGUAY: Tools Needed for Those Most Vulnerabl...         1    0.8087
..                                                 ...       ...       ...
135  Report: Save the Whales and They'll Save Us fr...         1     0.823
137  Arctic Beauty in Black and White: Alaska Befor...         1         1
138  #EPA report documents "very real" impacts from...         1         1
139  #Canadian #CEOs more keen on #green than globa...         1    0.7896
140  RT @carbonmarket: Ask the G8 & G20 to support ...         1     0.617

[100 rows x 3 columns]


<ipython-input-11-28bb6459ddde>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mensajesTwitter['CREENCIA'] = (mensajesTwitter['CREENCIA']=='Yes').astype(int)


Ahora, podemos crear una función llamada normalización que normalizará las frases.
Esto significa que la función permitirá la investigación de caracteres específicos, para que luego podamos tratarlos en un proceso específico.

La función re cambiará esos caracteres especiales a caracteres normales. Por ejemplo, un carácter de enlace se cambiará a una sola palabra 'URL'
También cambiará los acentos y las mayúsculas.

In [12]:
import re
def normalizacion(mensaje):
    mensaje = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', mensaje)
    mensaje = re.sub('@[^\s]+','USER', mensaje)
    mensaje = mensaje.lower().replace("ё", "е")
    mensaje = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', mensaje)
    mensaje = re.sub(' +',' ', mensaje)
    return mensaje.strip()

Aplicamos la función

In [13]:
mensajesTwitter["TWEET"] = mensajesTwitter["TWEET"].apply(normalizacion)
print(mensajesTwitter.head(10))

                                                TWEET  CREENCIA CONFIANZA
1   global warming report urges governments to act...         1         1
2   fighting poverty and global warming in africa ...         1         1
3   carbon offsets how a vatican forest failed to ...         1    0.8786
4   carbon offsets how a vatican forest failed to ...         1         1
5   uruguay tools needed for those most vulnerable...         1    0.8087
6   rt user rt user ocean saltiness shows global w...         1         1
7   global warming evidence all around us a messag...         1         1
8   migratory birds new climate change strategy st...         1         1
9   southern africa competing for limpopo water cl...         1         1
10  global warming to impact wheat rice production...         1         1


<ipython-input-13-d931aa831d20>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mensajesTwitter["TWEET"] = mensajesTwitter["TWEET"].apply(normalizacion)


Ahora, comenzamos a tratar las palabras vacías que tenemos en inglés

In [14]:
from nltk.corpus import stopwords
stopWords = stopwords.words('english')

Ahora que sabemos cuáles son las palabras vacías, podemos eliminarlas:

In [15]:
mensajesTwitter['TWEET'] = mensajesTwitter['TWEET'].apply(lambda mensaje: ' '.join([palabra for palabra in mensaje.split() if palabra not in (stopWords)]))
print(mensajesTwitter.head(10))

                                                TWEET  CREENCIA CONFIANZA
1   global warming report urges governments act br...         1         1
2         fighting poverty global warming africa link         1         1
3   carbon offsets vatican forest failed reduce gl...         1    0.8786
4   carbon offsets vatican forest failed reduce gl...         1         1
5   uruguay tools needed vulnerable climate change...         1    0.8087
6   rt user rt user ocean saltiness shows global w...         1         1
7   global warming evidence around us message glob...         1         1
8   migratory birds new climate change strategy st...         1         1
9   southern africa competing limpopo water climat...         1         1
10  global warming impact wheat rice production in...         1         1


<ipython-input-15-57bbf70baf03>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mensajesTwitter['TWEET'] = mensajesTwitter['TWEET'].apply(lambda mensaje: ' '.join([palabra for palabra in mensaje.split() if palabra not in (stopWords)]))


Aplicación de stemming

In [16]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
mensajesTwitter['TWEET'] = mensajesTwitter['TWEET'].apply(lambda mensaje: ' '.join([stemmer.stem(palabra) for palabra in mensaje.split(' ')]))
print(mensajesTwitter.head(10))

                                                TWEET  CREENCIA CONFIANZA
1   global warm report urg govern act brussel belg...         1         1
2               fight poverti global warm africa link         1         1
3   carbon offset vatican forest fail reduc global...         1    0.8786
4   carbon offset vatican forest fail reduc global...         1         1
5          uruguay tool need vulner climat chang link         1    0.8087
6   rt user rt user ocean salti show global warm i...         1         1
7   global warm evid around us messag global warm ...         1         1
8   migratori bird new climat chang strategi stay ...         1         1
9   southern africa compet limpopo water climat ch...         1         1
10  global warm impact wheat rice product india lu...         1         1


<ipython-input-16-6bad69e7505d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mensajesTwitter['TWEET'] = mensajesTwitter['TWEET'].apply(lambda mensaje: ' '.join([stemmer.stem(palabra) for palabra in mensaje.split(' ')]))


Lematización

In [17]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
mensajesTwitter['TWEET'] = mensajesTwitter['TWEET'].apply(lambda mensaje: ' '.join([lemmatizer.lemmatize(palabra) for palabra in mensaje.split(' ')]))
print(mensajesTwitter.head(10))

                                                TWEET  CREENCIA CONFIANZA
1   global warm report urg govern act brussel belg...         1         1
2               fight poverti global warm africa link         1         1
3   carbon offset vatican forest fail reduc global...         1    0.8786
4   carbon offset vatican forest fail reduc global...         1         1
5          uruguay tool need vulner climat chang link         1    0.8087
6   rt user rt user ocean salti show global warm i...         1         1
7   global warm evid around u messag global warm d...         1         1
8   migratori bird new climat chang strategi stay ...         1         1
9   southern africa compet limpopo water climat ch...         1         1
10  global warm impact wheat rice product india lu...         1         1


<ipython-input-17-0510f4d08012>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mensajesTwitter['TWEET'] = mensajesTwitter['TWEET'].apply(lambda mensaje: ' '.join([lemmatizer.lemmatize(palabra) for palabra in mensaje.split(' ')]))


Una vez que hemos hecho la preparación de las fases, estamos listos para mostrarle a nuestro algoritmo cómo tratar la información.

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mensajesTwitter['TWEET'].values,  mensajesTwitter['CREENCIA'].values,test_size=0.2)

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

etapas_aprendizaje = Pipeline([('frequencia', CountVectorizer()),
                                  ('tfidf', TfidfTransformer()),
                                  ('algoritmo', MultinomialNB())])

Aprendizaje

In [20]:
modelo = etapas_aprendizaje.fit(X_train,y_train)

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test, modelo.predict(X_test), digits=4))

              precision    recall  f1-score   support

           0     0.8421    0.2909    0.4324       220
           1     0.7971    0.9808    0.8795       625

    accuracy                         0.8012       845
   macro avg     0.8196    0.6359    0.6560       845
weighted avg     0.8088    0.8012    0.7631       845



In [22]:
frase = "Why should trust scientists with global warming if they didnt know Pluto wasnt a planet"
frase

'Why should trust scientists with global warming if they didnt know Pluto wasnt a planet'

Una vez que hemos leído la frase, lo primero que vamos a hacer es normalizarla.

In [23]:
#Normalización
frase = normalizacion(frase)

In [24]:
#Eliminación de las stops words
frase = ' '.join([palabra for palabra in frase.split() if palabra not in (stopWords)])

In [25]:
#Aplicación de stemming
frase =  ' '.join([stemmer.stem(palabra) for palabra in frase.split(' ')])

In [26]:
#Lematización
frase = ' '.join([lemmatizer.lemmatize(palabra) for palabra in frase.split(' ')])
print (frase)

trust scientist global warm didnt know pluto wasnt planet


In [27]:
prediccion = modelo.predict([frase])
print(prediccion)
if(prediccion[0]==0):
    print(">> No cree en el calentamiento climático...")
else:
    print(">> Cree en el calentamiento climático...")

[0]
>> No cree en el calentamiento climático...


Ahora, quiero asegurarme de que este algoritmo funcione. Así que usaré una frase que apoye el calentamiento global para ver si la predicción final es correcta.

In [28]:
frase = 'Climate change is real, it is happening now. It is the most urgent threat facing the different species and we need to work together and not leave things for later'
frase

'Climate change is real, it is happening now. It is the most urgent threat facing the different species and we need to work together and not leave things for later'

In [29]:
#Normalización
frase = normalizacion(frase)

In [30]:
#Eliminación de las stops words
frase = ' '.join([palabra for palabra in frase.split() if palabra not in (stopWords)])

In [31]:
#Aplicación de stemming
frase =  ' '.join([stemmer.stem(palabra) for palabra in frase.split(' ')])

In [32]:
#Lematización
frase = ' '.join([lemmatizer.lemmatize(palabra) for palabra in frase.split(' ')])
print (frase)

climat chang real happen urgent threat face differ speci need work togeth leav thing later


In [33]:
prediccion = modelo.predict([frase])
print(prediccion)
if(prediccion[0]==0):
    print(">> No cree en el calentamiento climático...")
else:
    print(">> Cree en el calentamiento climático...")

[1]
>> Cree en el calentamiento climático...


Uso de SVM para el mismo proceso

In [34]:
#Definición de la canalización
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
etapas_aprendizaje = Pipeline([('frequencia', CountVectorizer()),
                                  ('tfidf', TfidfTransformer()),
                                  ('algoritmo', svm.SVC(kernel='linear', C=2))])


In [35]:
#Aprendizaje
modelo = etapas_aprendizaje.fit(X_train,y_train)
from sklearn.metrics import classification_report
print(classification_report(y_test, modelo.predict(X_test), digits=4))

              precision    recall  f1-score   support

           0     0.7143    0.5909    0.6468       220
           1     0.8643    0.9168    0.8898       625

    accuracy                         0.8320       845
   macro avg     0.7893    0.7539    0.7683       845
weighted avg     0.8252    0.8320    0.8265       845



In [36]:
#Búsqueda del mejor parámetro C
from sklearn.model_selection import GridSearchCV
parametrosC = {'algoritmo__C':(1,2,4,5,6,7,8,9,10,11,12)}

In [37]:
busquedaCOptimo = GridSearchCV(etapas_aprendizaje, parametrosC,cv=2)
busquedaCOptimo.fit(X_train,y_train)
print(busquedaCOptimo.best_params_)

{'algoritmo__C': 2}


In [38]:
#Parámetro nuevo C=1
etapas_aprendizaje = Pipeline([('frequencia', CountVectorizer()),
                                  ('tfidf', TfidfTransformer()),
                                  ('algoritmo', svm.SVC(kernel='linear', C=1))])

In [39]:
modelo = etapas_aprendizaje.fit(X_train,y_train)
from sklearn.metrics import classification_report
print(classification_report(y_test, modelo.predict(X_test), digits=4))

              precision    recall  f1-score   support

           0     0.7368    0.5727    0.6445       220
           1     0.8605    0.9280    0.8930       625

    accuracy                         0.8355       845
   macro avg     0.7987    0.7504    0.7687       845
weighted avg     0.8283    0.8355    0.8283       845

